In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 4.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [164]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [165]:
df= pd.read_csv('data/men_shoes.csv')

def run_model (feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df [feats].values
  y = df ['prices_amountmin'].values

  #model = DecisionTreeRegressor (max_depth=5)

  scores = cross_val_score (model, X, y, scoring='neg_mean_absolute_error')
  return np.mean (scores), np.std(scores)  

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [166]:


df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])


(-58.133398968282776, 4.206122611474276)

In [167]:
model = RandomForestRegressor (max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [168]:
df.features.head().values 

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [169]:
 #zmienimy string na słownik

str_dirc = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dirc)
literal_eval(str_dirc)[0]['value'][0]

'Men'

In [0]:


#[{'key': 'Gender', 'value': ['Men']}, 
# {'key': 'Shoe Size', 'value': ['M']}, 
# {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, 
# {'key': 'Color', 'value': ['Multicolor']}, 
# {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, 
# {'key': 'Brand', 'value': ['Josmo']}]

 #chcemy zrobić w funkcji parse

 #'Gender':'Men' 
 #'Shoe Size': 'M'

 


In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"', '"'))
 
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

In [0]:
df['features_parsed'] = df['features'].map(parse_features)

In [175]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len (keys)

476

In [176]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [177]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan )

In [178]:
df.shape[0]

18280

In [179]:
df [df['feat_athlete'].isnull()].shape

(18272, 526)

In [180]:
df [ False == df['feat_athlete'].isnull()].shape[0]

8

In [181]:
 df [ False == df['feat_athlete'].isnull()].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
keys_stat = {}

for key in keys: 
  keys_stat [key] = df [ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100


In [0]:
#keys_stat

In [188]:
{k:v for k, v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]
df['feat_picture_cat'] = df['feat_picture'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


In [219]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() ) 
df [ df.brand == df.feat_brand ].shape
#df [ df.brand != df.feat_brand ] [['brand', 'feat_brand']].head()

(8846, 1002)

In [220]:
feats = ['']


model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.298910792804826, 4.209981703462378)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]
#feats_cat

In [0]:
feats = ['brand_cat', 'feat_brand_cat',  'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat', 'feat_picture_cat'] #'feat_manufacturer part number_cat',  #'feat_color_cat',
#feats += feats_cat
#feats = list(set(feats))

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model) 

In [231]:
 X = df[ feats].values
 y = df['prices_amountmin'].values
 
 m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
 m.fit(X,y)

print(result)
 perm = PermutationImportance (m, random_state=1).fit(X,y)
 eli5.show_weights(perm, feature_names=feats)

(-57.0195280545219, 4.223011689422417)


Weight,Feature
0.2621 ± 0.0110,brand_cat
0.1053 ± 0.0073,feat_material_cat
0.0427 ± 0.0023,feat_gender_cat
0.0212 ± 0.0005,feat_brand_cat
0.0071 ± 0.0009,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat
0 ± 0.0000,feat_picture_cat


In [207]:
df ['brand'].value_counts(normalize=True)
df ['brand'].value_counts()

nike               1777
puma                609
ralph lauren        526
vans                386
new balance         371
                   ... 
key apparel           1
chalktalksports       1
athletic works        1
saddle tramp          1
z-coil                1
Name: brand, Length: 1732, dtype: int64

In [214]:
df [df['brand'] == 'nike'].features_parsed.head().values
df [df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'gender': 'men', 'brand': 'nike'}, {},
       {'gender': 'men', 'shoe category': 'shoes', 'brand': 'nike', 'manufacturer_part_number': '705178-045', 'fabric content': '100% leather', 'color': 'dust/metallic gold-black-retro'},
       {'type': 'cleats', 'condition': 'new with box'},
       {'sport': 'running shoes', 'occasion': 'running shoes', 'material': 'mesh', 'gender': 'men', 'shoe size': '8 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'mesh', 'color': 'black', 'model': '819300 005', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '819300 005', 'brand': 'nike', 'shoe width': 'd(m)'}],
      dtype=object)

In [238]:
 !git add day5.ipynb

fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [239]:
cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [240]:
 !git add day5.ipynb

fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
